In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("akhooli/xlm-r-large-arabic-toxic")

model = AutoModelForSequenceClassification.from_pretrained("akhooli/xlm-r-large-arabic-toxic")




In [4]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install git+https://github.com/huggingface/transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hml132vw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hml132vw
  Resolved https://github.com/huggingface/transformers to commit c5454eba9eac00a3e7d0a46a3d25aacd43187f1e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!pip install --upgrade pip
!pip install --upgrade setuptools wheel
!pip install tokenizers
!pip install transformers==4.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.6.1-py3-none-any.whl (2.2 MB)
  Using cached huggingface_hub-0.0.8-py3-none-any.whl (34 kB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with 

In [7]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('CompanyReviews.csv')
df = df.dropna()
df.head()

,Unnamed: 0,review_description,rating,company
0,0,رائع,1,talbat
1,1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat
2,2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat
3,3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat
4,4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat


In [8]:
#converting the column values to string so we can split them to diffirent columns.
#df[["label", "tweet"]] = df["label\ttext"].str.split("\t", expand = True)
#removing the original column.
#df = df.drop(columns = ["label\ttext"])
#df.head()

In [9]:
print(df['rating'].unique())
print(df['rating'].value_counts()[0])
# Map the class labels to integers
label_map = {1: 0, -1: 1}
df['rating'] = df['rating'].map(label_map)
df.head()

[ 1 -1  0]
1925


<ipython-input-9-ed92b209bb3a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].map(label_map)


,Unnamed: 0,review_description,rating,company
0,0,رائع,0.0,talbat
1,1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,0.0,talbat
2,2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,1.0,talbat
3,3,لماذا لا يمكننا طلب من ماكدونالدز؟,1.0,talbat
4,4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,1.0,talbat


In [10]:
from math import nan
import numpy as np
num_nan = df['rating'].isna().sum()


print(f'Number of rows with NaN values in the "rating" column: {num_nan}')

df.dropna(subset=['rating'], inplace=True)

print(f'Number of rows with NaN values in the "rating" column: {num_nan}')


Number of rows with NaN values in the "rating" column: 1925
Number of rows with NaN values in the "rating" column: 1925


<ipython-input-10-b344a40a6996>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['rating'], inplace=True)


In [11]:
print(df['rating'].unique())

[0. 1.]


In [12]:
from transformers import InputExample, InputFeatures

input_texts = df['review_description'].astype(str).values.tolist()

# Tokenize the input sequences using the tokenizer you loaded
encoded_data = tokenizer.batch_encode_plus(
    input_texts,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt')


In [13]:
# Split the data into training, validation, and test sets
train_size = int(0.8 * len(df))
val_size = int(0.1 * len(df))
test_size = len(df) - train_size - val_size

In [14]:
train_data = [{'input': {'input_ids': input_ids, 'attention_mask': attention_mask}, 'label': labels}
              for input_ids, attention_mask, labels in zip(encoded_data['input_ids'][:train_size],
                                                           encoded_data['attention_mask'][:train_size],
                                                           df['rating'][:train_size])]
val_data = [{'input': {'input_ids': input_ids, 'attention_mask': attention_mask}, 'label': label}            for input_ids, attention_mask, label in zip(encoded_data['input_ids'][train_size:train_size+val_size],
                                                         encoded_data['attention_mask'][train_size:train_size+val_size],
                                                         df['rating'][train_size:train_size+val_size])]
test_data = [{'input': {'input_ids': input_ids, 'attention_mask': attention_mask}, 'label': label}             for input_ids, attention_mask, label in zip(encoded_data['input_ids'][-test_size:],
                                                          encoded_data['attention_mask'][-test_size:],
                                                          df['rating'][-test_size:])]

In [15]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
# Define your model architecture
class MyModel(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained("akhooli/xlm-r-large-arabic-toxic", num_labels=num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits


num_labels = 2  # Specify the number of labels
model = MyModel(num_labels)

In [16]:
# Define the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define the optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
num_epochs = 3
batch_size = 100
num_train_steps = len(train_data) * num_epochs // batch_size
num_warmup_steps = num_train_steps // 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
num_batches = len(train_data) // batch_size

print("Number of batches:", num_batches)

Number of batches: 304


In [17]:
# Define the evaluation function
def eval_step(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input']['input_ids'].to(device)
            attention_mask = batch['input']['attention_mask'].to(device)
            labels = batch['label'].to(device).long()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return accuracy

In [18]:
# Training loop
for epoch in range(num_epochs):
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

    model.train()
    for i, batch in enumerate(train_dataloader):
        input_ids = batch['input']['input_ids'].to(device)
        attention_mask = batch['input']['attention_mask'].to(device)
        labels = batch['label'].to(device).long()

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        if i % 100 == 0:
            train_acc = eval_step(model, train_dataloader)
            val_acc = eval_step(model, val_dataloader)
            print(f'Epoch: {epoch+1}, Batch: {i+1}/{len(train_dataloader)}, Train Acc: {train_acc:.3f}, Val Acc: {val_acc:.3f}')



    # Evaluate model on validation data after each epoch
    val_acc = eval_step(model, val_dataloader)
    print(f'Epoch: {epoch+1}, Val Acc: {val_acc:.3f}')




Epoch: 1, Batch: 1/305, Train Acc: 0.692, Val Acc: 0.658
Epoch: 1, Batch: 101/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 1, Batch: 201/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 1, Batch: 301/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 1, Val Acc: 0.589
Epoch: 2, Batch: 1/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 2, Batch: 101/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 2, Batch: 201/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 2, Batch: 301/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 2, Val Acc: 0.589
Epoch: 3, Batch: 1/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 3, Batch: 101/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 3, Batch: 201/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 3, Batch: 301/305, Train Acc: 0.650, Val Acc: 0.589
Epoch: 3, Val Acc: 0.589


In [19]:
# Evaluate model on the test set
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
test_acc = eval_step(model, test_dataloader)
print(f'Test Acc: {test_acc:.3f}')

Test Acc: 0.490


In [20]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set your model in evaluation mode
model.eval()

# Initialize lists to store predictions and labels
val_predictions = []
val_labels = []
test_predictions = []
test_labels = []

# Evaluation on validation set
with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input']['input_ids'].to(device)
        attention_mask = batch['input']['attention_mask'].to(device)
        labels = batch['label'].to(device).long()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs, dim=1)

        # Append predictions and labels
        val_predictions.extend(predicted_labels.tolist())
        val_labels.extend(labels.tolist())

# Evaluation on testing set
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input']['input_ids'].to(device)
        attention_mask = batch['input']['attention_mask'].to(device)
        labels = batch['label'].to(device).long()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs, dim=1)

        # Append predictions and labels
        test_predictions.extend(predicted_labels.tolist())
        test_labels.extend(labels.tolist())

# Calculate evaluation metrics
val_accuracy = accuracy_score(val_labels, val_predictions)
val_precision = precision_score(val_labels, val_predictions, average='weighted')
val_recall = recall_score(val_labels, val_predictions, average='weighted')
val_f1 = f1_score(val_labels, val_predictions, average='weighted')

test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average='weighted')
test_recall = recall_score(test_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_labels, test_predictions, average='weighted')

# Print evaluation results
print("Validation Set:")
print("Accuracy: {:.3f}".format(val_accuracy))
print("Precision: {:.3f}".format(val_precision))
print("Recall: {:.3f}".format(val_recall))
print("F1 Score: {:.3f}".format(val_f1))
print()
print("Testing Set:")
print("Accuracy: {:.3f}".format(test_accuracy))
print("Precision: {:.3f}".format(test_precision))
print("Recall: {:.3f}".format(test_recall))
print("F1 Score: {:.3f}".format(test_f1))


Validation Set:
Accuracy: 0.589
Precision: 0.347
Recall: 0.589
F1 Score: 0.436

Testing Set:
Accuracy: 0.490
Precision: 0.240
Recall: 0.490
F1 Score: 0.322


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
